In [3]:
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime
import pandas.io.sql as sqlio
import csv


# Importar os CSV's

In [9]:
# read data
df = pd.read_csv('CSV Files/books.csv')
#df = pd.read_csv("/Users/henriqueribeiro/Downloads/ad-goodreads-project-main/books.csv", delimiter=';', low_memory=False)
cols_to_drop = ['average_rating', 'ratings_1','ratings_2','ratings_3','ratings_4','ratings_5']
df.drop(cols_to_drop, inplace=True, axis=1)




In [10]:
df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,original_title,title,language_code,ratings_count,work_ratings_count,work_text_reviews_count
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4780653,4942365,155254
1,2,3,3,4640799,491,439554934,9.780440e+12,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4602479,4800065,75867
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Twilight,"Twilight (Twilight, #1)",en-US,3866839,3916824,95009
3,4,2657,2657,3275794,487,61120081,9.780061e+12,To Kill a Mockingbird,To Kill a Mockingbird,eng,3198671,3340896,72586
4,5,4671,4671,245494,1356,743273567,9.780743e+12,The Great Gatsby,The Great Gatsby,eng,2683664,2773745,51992


In [11]:
tags = pd.read_csv('CSV Files/tags.csv')


In [12]:
tags.head(5)

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


# Conectar à DB

In [170]:
## connect to database
con = psycopg2.connect(
    database="fced_henrique_ribeiro",             # your database is the same as your username
    user="fced_henrique_ribeiro",                 # your username
    password="feup_dbm00",             # your password
    host="dbm.fe.up.pt",
    port="5433",             # the database host
    options='-c search_path=goodreads'  # use the schema you want to connect to
)

# Criação das Tabelas

In [70]:
cur = con.cursor()
print('Connetion established!')



#Criação das tabelas
cur.execute('''CREATE TABLE book(
book_id INTEGER PRIMARY KEY UNIQUE, 
goodreads_book_id INTEGER UNIQUE,
best_book_id INTEGER UNIQUE,
work_id INTEGER UNIQUE,
books_count INTEGER,
isbn NUMERIC(15,2),
isbn13 NUMERIC(15,2),
original_title VARCHAR(255),
title VARCHAR(255),
language_code VARCHAR(255));
''')
print("Table book created successfully")





Connetion established!


DuplicateTable: relation "book" already exists


In [ ]:
#Criação da tabela tags
cur.execute('''CREATE TABLE tag(
    tag_id INTEGER PRIMARY KEY UNIQUE,
    tag_name VARCHAR(255));
''')
print("Table tag created successfully")


In [ ]:
#Criação da tabela books_tags

cur.execute('''CREATE TABLE book_tags(
    goodreads_book_id INTEGER REFERENCES book(goodreads_book_id),
    tag_id INTEGER REFERENCES tag(tag_id),
    count INTEGER;''')
print("Table book_tags created successfully")


In [ ]:
#criação da tabela authors
cur.execute('''CREATE TABLE author(
    author_id INTEGER PRIMARY KEY UNIQUE,
    name VARCHAR(255) UNIQUE
    Address VARCHAR(255),
    Sex VARCHAR(255),
    email VARCHAR(255),
    );''')
print("Table author created successfully")

In [ ]:
#criação da tabela books_authors
cur.execute('''CREATE TABLE authorship(
    book_id INTEGER REFERENCES book(book_id),
    author_id INTEGER REFERENCES author(author_id),
    publication_year INTEGER
    );''')
print('Table authorship created successfully')

In [ ]:
#criação da tabela users

cur.execute('''CREATE TABLE user(
    user_id INTEGER PRIMARY KEY UNIQUE,
    sex VARCHAR(255),
    name VARCHAR(255) UNIQUE,
    email VARCHAR(255),
    address VARCHAR(255),
    phone_number INTEGER,
    );''')
print('Table user created successfully')


In [ ]:
#criação da tabela to_read

cur.execute('''CREATE TABLE to_read(
    user_id INTEGER REFERENCES user(user_id),
    book_id INTEGER REFERENCES book(book_id),
    time TIMESTAMP);''')

print('Table to_read created successfully')

# Popular as tabelas

In [171]:
cur = con.cursor()
print('Connetion established!')

Connetion established!


In [105]:
with open('/Users/henriqueribeiro/Downloads/ad-goodreads-project-main/tags.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'tag', sep=';', columns=('tag_id', 'tag_name'))
con.commit()
print("Table tag loaded successfully")


Table tag loaded successfully


In [172]:
#load data into table book
with open('/Users/henriqueribeiro/Downloads/ad-goodreads-project-main/books.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'book', sep=';', columns=('book_id', 'goodreads_book_id','best_book_id','work_id','books_count','isbn','isbn13','original_title','title','language_code'))
con.commit()

print("Table book loaded successfully")




Table book loaded successfully


In [173]:
#load data into table book_tags
with open('/Users/henriqueribeiro/Downloads/ad-goodreads-project-main/book_tags.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'book_tags', sep=';', columns=('goodreads_book_id', 'tag_id','count'))
con.commit()

print("Table book_tags loaded successfully")

Table book_tags loaded successfully


In [169]:

#close the connection
con.close()

In [ ]:


# now = datetime.now() 
# print("Time Started =", now)


# read data
# df = pd.read_csv("/Users/henriqueribeiro/Desktop/AD - Projeto/ad-goodreads-project/.books.csv.icloud", low_memory=False)
# df.drop_duplicates(inplace=True, ignore_index=True)
# cols_to_drop = ['average_rating', 'rating_1','rating_2,','rating_3','rating_4','rating_5','image_url','small_image_url']


# # def trf(offtime_):
# #     position = offtime_.find("s")
# #     offtime = offtime_[position + 2:]
# #     position = offtime.find(".")
# #     offtime = offtime[:position]
# #     offtime = datetime.strptime(offtime, '%H:%M:%S')
# #     return offtime
# #df = df.where(pd.notnull(df), None)
# df["official_time"]=df["official_time"].apply(trf)
# df["official_time"]=pd.to_datetime(df["official_time"], format='%H:%M:%S')
# df["net_time"]=df["net_time"].apply(lambda x: trf(x) if isinstance(x, str) else None)
# df['net_time']=df['net_time'].replace({np.nan: None}, inplace = True)
# df["official_time"]=pd.to_datetime(df["official_time"], format='%H:%M:%S')
# df['birth_date']=df['birth_date'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y"))
# df['distance'] = df['distance'].astype(int)


# # prepare data
# teams = pd.DataFrame(df['team'].dropna().unique(), dtype="string")
# teams = teams.dropna()
# #print(teams)

#teams = teams[~np.isnan(teams)]
# runners = df[["name", "birth_date", "sex", "nation"]]
# runners = runners.drop_duplicates(ignore_index=True)
# events = df[["event", "distance"]]
# events = events.drop_duplicates(ignore_index=True)




cur = con.cursor()
print('connetion established')
## delete current entries
# cur.execute("TRUNCATE event CASCADE")
# cur.execute("TRUNCATE runner CASCADE")
# cur.execute("TRUNCATE team CASCADE")
# cur.execute("TRUNCATE race")
# con.commit()

# print('Tables empty')
# load teams
book_id = df['book_id']
with cur.copy("COPY team(team_name) FROM STDIN") as copy:
    for b_id in book_id:
        copy.write_row(book_id)
con.commit()
print("Book ID's loaded")

#events

events = list(events.itertuples(index=False, name=None))
with cur.copy("COPY event(event, distance) FROM STDIN") as copy:
    for e in events:
        copy.write_row(e)
con.commit()
print("Events loaded")

#runners
runners = list(runners.itertuples(index=False, name=None))
with cur.copy("COPY runner(name, birth_date, sex, nation) FROM STDIN") as copy:
    for r in runners:
        copy.write_row(r)
con.commit()
print("Runners loaded")

#races
eventr = sqlio.read_sql_query("SELECT * FROM event", con)
eventr['distance'] = eventr['distance'].astype(int)
runnerr = sqlio.read_sql_query("SELECT * FROM runner", con)
teamr  = sqlio.read_sql_query("SELECT * FROM team", con)
runnerr['birth_date']=pd.to_datetime(runnerr['birth_date'])
r_merge = pd.merge(df, runnerr, how = 'left', on = ['name', 'birth_date', 'nation', 'sex'])
event_merge = pd.merge(r_merge, eventr, how = 'left', on = ['event', 'distance'])
team_merge = pd.merge(event_merge, teamr, how = 'left', left_on='team', right_on='team_name') 
team_merge['o_time'] = team_merge['official_time']
team_merge['n_time'] = team_merge['net_time']
team_merge = team_merge.where(pd.notnull(team_merge), None)
races = team_merge[['e_id', 'ru_id', 'team_name', 'event_year', 'bib', 'age_class', 'place', 'place_in_class', 'o_time', 'n_time']]

races = list(races.itertuples(index=False, name=None))
with cur.copy("COPY race(e_id, ru_id, team_name, event_year, bib, age_class, place, place_in_class, o_time, n_time) FROM STDIN") as copy:
    for r in races:
        copy.write_row(r)
con.commit()
print("Races loaded")
con.close()
now = datetime.now() 
print("Time Ended =", now)